# Challenge 3

In [1]:
!pip install datasets
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install evaluate
!pip install trl
!pip install rouge_score

/bin/bash: line 1: pin: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fss

In [2]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from huggingface_hub import notebook_login
import evaluate
from trl import SFTTrainer
import requests
import json

In [3]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Get the number of available CUDA devices
    num_devices = torch.cuda.device_count()
    print("Number of available CUDA devices:", num_devices)

    # Iterate over CUDA devices and print their indices and names
    for i in range(num_devices):
        print("GPU index", i, ":", torch.cuda.get_device_name(i))
else:
    print("CUDA is not available. CPU will be used.")

Number of available CUDA devices: 1
GPU index 0 : Tesla T4


In [4]:
#notebook_login()

In [5]:
# Base model
# Llama-2-7b-hf model architecture: It is an auto-regressive language model that uses an optimized transformer architecture.
# It is possible to the official Meta Llama-2 model from Hugging Face, but you have to apply and wait a couple of days for confirmation.
# Model: https://huggingface.co/meta-llama/Llama-2-7b-hf
# Paper: https://arxiv.org/abs/2307.09288
# More info: https://llama.meta.com/llama2/

#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

In [6]:
# Base model
# Model: https://huggingface.co/meta-llama/Meta-Llama-3-8B

In [7]:
# Dataset
# Train
url_train = "https://raw.githubusercontent.com/architkaila/Fine-Tuning-LLMs-for-Medical-Entity-Extraction/main/data/entity_extraction/entity-extraction-train-data.json"
response_train = requests.get(url_train)
if response_train.status_code == 200:
    data_train = response_train.json()
    print("Training OK")
else:
    print("Error obtaining training data:", response_train.status_code)

Training OK


In [8]:
data_train[0]

{'input': "Robert Johnson\nrobert.johnson@email.com\n789 Maple Lane, Chicago, IL 60601\n555-234-5678, United States\n\nRelationship to XYZ Pharma Inc.: Patient\nReason for contacting: Adverse Event\n\nMessage: I've been on Onglyza for a while, and I've noticed that I'm experiencing frequent painful urination. Is this a known side effect?",
 'output': '{"drug_name": "Onglyza", "adverse_events": ["painful urination"]}'}

In [9]:
# Test
url_test = "https://raw.githubusercontent.com/architkaila/Fine-Tuning-LLMs-for-Medical-Entity-Extraction/main/data/entity_extraction/entity-extraction-test-data.json"
response_test = requests.get(url_test)
if response_test.status_code == 200:
    data_test = response_test.json()
    print("Test OK")
else:
    print("Error obtaining test data:", response_test.status_code)

Test OK


In [10]:
data_test[0]

{'input': "Natalie Cooper,\nncooper@example.com\n6789 Birch Street, Denver, CO 80203,\n303-555-6543, United States\n\nRelationship to XYZ Pharma Inc.: Patient\nReason for contacting: Adverse Event\n\nMessage: Hi, after starting Abilify for bipolar I disorder, I've noticed that I am experiencing nausea and vomiting. Are these typical reactions? Best, Natalie Cooper",
 'output': '{"drug_name": "Abilify", "adverse_events": ["nausea", "vomiting"]}'}

```
---- LlaMa2 datasets ----
https://gpus.llm-utils.org/llama-2-prompt-template/
Note that this only applies to the llama 2 chat models. The base models have no prompt structure, they’re raw non-instruct tuned models.

<s>[INST] {user_message_1} [/INST] {model_reply_1}</s>

---- Alpaca datasets ----
### Instruction:
(Instruction Text)

### Input:
(Auxiliary Input Text)

### Response:
(Desired Response Text)

---- Vicuna datasets ----
Vicuna datasets
### Human:
(Question Text)
### Assistant:
(Response Text)

---- Mistral datasets ----
<s>[INST] Instruction [/INST] Model answer</s>

---- Gemma ----
```

In [11]:
# Without a system message
# <s>[INST] {user_message_1} [/INST] {model_reply_1}</s>

formatted_data_train = []

for item in data_train:
    input_text = item["input"]
    output_text = item["output"]

    formatted_input = f"<s>[INST] {input_text}[/INST]"

    formatted_output = output_text.replace('\"', '').replace('{', '').replace('}', '')

    formatted_data_train.append({'text':formatted_input + formatted_output + '</s>'})

In [12]:
formatted_data_train[0]

{'text': "<s>[INST] Robert Johnson\nrobert.johnson@email.com\n789 Maple Lane, Chicago, IL 60601\n555-234-5678, United States\n\nRelationship to XYZ Pharma Inc.: Patient\nReason for contacting: Adverse Event\n\nMessage: I've been on Onglyza for a while, and I've noticed that I'm experiencing frequent painful urination. Is this a known side effect?[/INST]drug_name: Onglyza, adverse_events: [painful urination]</s>"}

In [13]:
dataset_train = Dataset.from_list(formatted_data_train)

In [14]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 700
})

In [15]:
dataset_train[0]

{'text': "<s>[INST] Robert Johnson\nrobert.johnson@email.com\n789 Maple Lane, Chicago, IL 60601\n555-234-5678, United States\n\nRelationship to XYZ Pharma Inc.: Patient\nReason for contacting: Adverse Event\n\nMessage: I've been on Onglyza for a while, and I've noticed that I'm experiencing frequent painful urination. Is this a known side effect?[/INST]drug_name: Onglyza, adverse_events: [painful urination]</s>"}

In [16]:
formatted_data_test = []

for item in data_test:
    input_text = item["input"]
    output_text = item["output"]

    formatted_input = f"<s>[INST] {input_text}[/INST]"

    formatted_output = output_text.replace('\"', '').replace('{', '').replace('}', '')

    formatted_data_test.append({'text':formatted_input + formatted_output + '</s>'})

In [17]:
formatted_data_test[0]

{'text': "<s>[INST] Natalie Cooper,\nncooper@example.com\n6789 Birch Street, Denver, CO 80203,\n303-555-6543, United States\n\nRelationship to XYZ Pharma Inc.: Patient\nReason for contacting: Adverse Event\n\nMessage: Hi, after starting Abilify for bipolar I disorder, I've noticed that I am experiencing nausea and vomiting. Are these typical reactions? Best, Natalie Cooper[/INST]drug_name: Abilify, adverse_events: [nausea, vomiting]</s>"}

In [18]:
dataset_test = Dataset.from_list(formatted_data_test) # HuggingFace Dataset

In [19]:
dataset_test

Dataset({
    features: ['text'],
    num_rows: 59
})

In [20]:
dataset_test[0]

{'text': "<s>[INST] Natalie Cooper,\nncooper@example.com\n6789 Birch Street, Denver, CO 80203,\n303-555-6543, United States\n\nRelationship to XYZ Pharma Inc.: Patient\nReason for contacting: Adverse Event\n\nMessage: Hi, after starting Abilify for bipolar I disorder, I've noticed that I am experiencing nausea and vomiting. Are these typical reactions? Best, Natalie Cooper[/INST]drug_name: Abilify, adverse_events: [nausea, vomiting]</s>"}

In [21]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf") # Load tokenizer
tokenizer.pad_token = tokenizer.eos_token #  Padding token of the tokenizer to be the same as the end-of-sequence (eos) token
tokenizer.padding_side = "right" # Padding should be added to the right side of the input sequences

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [22]:
# Base model
# We will use NousResearch's Llama-2-7b-chat-hf as a base model, which is the same as the original, but easily accessible.
# Model: https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

# Create quantization config
# https://huggingface.co/docs/transformers/main_classes/quantization
# Quantization techniques reduce memory and computational costs
# by representing weights and activations with lower-precision data types
quantization_config = BitsAndBytesConfig(
    Load_in_4bit=True, # This flag is used to enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16, # This sets the computational type: once the weights are loaded in 4-bit, the computations will be performed using 16-bit floating-point precision.
    bnb_4bit_quant_type="nf4" # This sets the quantization data type
)

model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf", # Load model
                                             quantization_config= quantization_config, # Quantification configuration
                                             device_map=0 # device_map = 0 means put the whole model on GPU 0; device_map="auto" compute the most optimized `device_map` automatically
)

Unused kwargs: ['Load_in_4bit']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [23]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

### Zero-shot

In [24]:
print(data_test[1]['input'])

Mia Garcia
mia.garcia@email.com
321 Magnolia Drive, Dallas, TX 75201
555-890-1234, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I experienced a feeling of light-headedness and near-fainting after taking Staxyn for my erectile dysfunction. Is this a common side effect, and should I be worried?


In [25]:
# Run text generation pipeline with our model
prompt = data_test[1]['input']
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] Mia Garcia
mia.garcia@email.com
321 Magnolia Drive, Dallas, TX 75201
555-890-1234, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I experienced a feeling of light-headedness and near-fainting after taking Staxyn for my erectile dysfunction. Is this a common side effect, and should I be worried? [/INST]  Subject: Adverse Event Report - Light-Headedness and Near-Fainting after Taking Staxyn

Dear Mia Garcia,

Thank you for reaching out to XYZ Pharma Inc. regarding your recent adverse event experience with Staxyn. We take all safety concerns seriously and appreciate


In [26]:
print(data_test[2]['input'])

Brandon Lee,
blee@example.com
3333 Pine Road, Hilltown, MA 02108,
617-555-3333, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: Since I started on Byetta, I've noticed an increase in thirst and dry mouth. Is this related to the medication? Best, Brandon Lee


In [27]:
# Run text generation pipeline with our model
prompt = data_test[2]['input']
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Brandon Lee,
blee@example.com
3333 Pine Road, Hilltown, MA 02108,
617-555-3333, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: Since I started on Byetta, I've noticed an increase in thirst and dry mouth. Is this related to the medication? Best, Brandon Lee [/INST]  Sure, here is an example of a reportable adverse event report for Byetta:

Patient: Brandon Lee

Contact Information:

* Email: blee@example.com
* Address: 3333 Pine Road, Hilltown, MA 02108
* Phone Number: 617-555-33


### One-shot

In [28]:
prompt = dataset_test[0]['text'] + '\n' + f"<s>[INST] {data_test[1]['input']} [/INST]"
print(prompt)

<s>[INST] Natalie Cooper,
ncooper@example.com
6789 Birch Street, Denver, CO 80203,
303-555-6543, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: Hi, after starting Abilify for bipolar I disorder, I've noticed that I am experiencing nausea and vomiting. Are these typical reactions? Best, Natalie Cooper[/INST]drug_name: Abilify, adverse_events: [nausea, vomiting]</s>
<s>[INST] Mia Garcia
mia.garcia@email.com
321 Magnolia Drive, Dallas, TX 75201
555-890-1234, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I experienced a feeling of light-headedness and near-fainting after taking Staxyn for my erectile dysfunction. Is this a common side effect, and should I be worried? [/INST]


In [29]:
# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=350)
result = pipe(prompt)
print(result[0]['generated_text'])

<s>[INST] Natalie Cooper,
ncooper@example.com
6789 Birch Street, Denver, CO 80203,
303-555-6543, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: Hi, after starting Abilify for bipolar I disorder, I've noticed that I am experiencing nausea and vomiting. Are these typical reactions? Best, Natalie Cooper[/INST]drug_name: Abilify, adverse_events: [nausea, vomiting]</s>
<s>[INST] Mia Garcia
mia.garcia@email.com
321 Magnolia Drive, Dallas, TX 75201
555-890-1234, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I experienced a feeling of light-headedness and near-fainting after taking Staxyn for my erectile dysfunction. Is this a common side effect, and should I be worried? [/INST]  drug_name: Staxyn, adverse_events: [lightheadedness, near-fainting]


### Training

In [30]:
# Create LoRA config
# More info in https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
peft_config = LoraConfig(
    r=8, # The rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.
    target_modules=["g_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # The modules to apply the LoRA update matrices.
    bias="none", # Specifies if the bias parameters should be trained.
    task_type = TaskType.CAUSAL_LM
)


In [31]:
# Subset of the arguments thath we use to the training.
# https://huggingface.co/docs/transformers/main_classes/trainer

training_params = TrainingArguments(
    output_dir="./results", # where the model's checkpoints and predictions will be stored
    num_train_epochs=1, # number of epochs
    per_device_train_batch_size=4, # batch size for training
    gradient_accumulation_steps=1, # # Number of update steps to accumulate the gradients for
    optim="paged_adamw_32bit", # AdamW optimizer
    save_steps=25, # save checkpoint every 25 update steps
    logging_steps=25, # logs every 25 update steps
    learning_rate=2e-4, # initial learning rate
    weight_decay=0.001, # weight decay to apply to all layers except bias/LayerNorm weights
    fp16=False,
    bf16=False,
    max_grad_norm=0.3, # maximum gradient normal (gradient clipping)
    max_steps=-1, # number of training steps (if not -1 overrides num_train_epochs)
    warmup_ratio=0.03, # ratio of steps for a linear warmup (from 0 to learning rate)
    group_by_length=True, # group sequences into batches with same length
    lr_scheduler_type="constant", # learning rate schedule
    report_to="tensorboard"
)

In [ ]:
# Set supervised fine-tuning parameters
max_seq_length = None
packing = False

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_params,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Step,Training Loss
25,1.343800
50,0.621500


In [ ]:
# Run text generation pipeline with our next model
prompt = data_test[1]['input']
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] Mia Garcia
mia.garcia@email.com
321 Magnolia Drive, Dallas, TX 75201
555-890-1234, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I experienced a feeling of light-headedness and near-fainting after taking Staxyn for my erectile dysfunction. Is this a common side effect, and should I be worried? [/INST]drug_name: Staxyn, adverse_events: [light-headedness, near-fainting]</s>


In [ ]:
# Run text generation pipeline with our next model
prompt = data_test[12]['input']
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Daniel Wilson
daniel.wilson@example.com
112 Pine Avenue, Atlanta, GA 30301
4045554321, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I took Nexium for acid reflux, and I had a headache and stomach pain. Could this be due to the medication? [/INST]drug_name: Nexium, adverse_events: [headache, stomach pain]</s>


### Evaluation

In [ ]:
# Evaluate the Model Quantitatively
rouge = evaluate.load('rouge') # https://en.wikipedia.org/wiki/ROUGE_(metric)

In [ ]:
input = []

for d in data_test:
  input.append(f"<s>[INST] {d['input']} [/INST]")

In [ ]:
output = dataset_test['text']

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

output_model = []

for i in input:
  output_model.append(pipe(i))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(output[10])

<s>[INST] William Harris
william.harris@example.com
890 Oak Road, San Francisco, CA 94101
4155558765, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I received Neupogen and had trouble breathing and fever. Are these common side effects of the medication?[/INST]drug_name: Neupogen, adverse_events: [trouble breathing, fever]</s>


In [ ]:
output_model__ = []

for ii in output_model:
  output_model__.append(ii[0]['generated_text'])

print(output_model__[10])

<s>[INST] William Harris
william.harris@example.com
890 Oak Road, San Francisco, CA 94101
4155558765, United States

Relationship to XYZ Pharma Inc.: Patient
Reason for contacting: Adverse Event

Message: I received Neupogen and had trouble breathing and fever. Are these common side effects of the medication? [/INST]drug_name: Neupogen, adverse_events: [trouble breathing, fever]</s>


In [ ]:
rouge_results = rouge.compute(
    predictions=output_model__,
    references=output,
    use_aggregator=True, # Scores are averaged over all examples
    use_stemmer=True, # Stemmer will be used during the computation of the ROUGE scores (stemmer reduces words to their root form, which can help in matching similar words)
)

In [ ]:
print(rouge_results)

{'rouge1': 0.9936390272974067, 'rouge2': 0.98509270552578, 'rougeL': 0.9922761109562539, 'rougeLsum': 0.9923333259355023}


In [ ]:
torch.cuda.empty_cache()

In [ ]:
total_mem = torch.cuda.get_device_properties(0).total_memory